In [3]:
import numpy as np

terrain_data = np.load(f'processed_terrain_data.npy')
terrain_labels = np.load(f'terrain_data_labels.npy')
terrain_columns = np.load('terrains_columns_metadata.npy')

# Clasificador SVM

In [4]:
import pickle
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


foldsNum = 5
kf = StratifiedKFold(n_splits=foldsNum, shuffle = True)
clf = svm.SVC(kernel = 'linear', class_weight='balanced')
acc = 0
n_classes = len(np.unique(terrain_labels))
recall = np.zeros(n_classes)
precision = np.zeros(n_classes)
for train_index, test_index in kf.split(terrain_data, terrain_labels):
    
    # Training phase
    x_train = terrain_data[train_index, :]
    y_train = terrain_labels[train_index]
    clf.fit(x_train, y_train)

    # Test phase
    x_test = terrain_data[test_index, :]
    y_test = terrain_labels[test_index]    
    y_pred = clf.predict(x_test)
    # Calculate confusion matrix and model performance
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion matrix\n', cm)

    accSum = 0
    for i in range(n_classes):
        accSum += cm[i,i]
    acc += accSum / len(y_test)

    for i in range(n_classes):
        innerSum = 0
        for y in range(n_classes):
            innerSum += cm[i,y]
        recall[i] += cm[i,i]/innerSum

    for i in range(n_classes):
        innerSum = 0
        for y in range(n_classes):
            innerSum += cm[y,i]
        precision[i] += cm[i,i]/innerSum

# Print results
acc = acc/foldsNum
print('Acc: ', acc)
precision = precision/foldsNum
print('Precision: ', precision)
recall = recall/foldsNum
print('Recall: ', recall)

with open(f'terrain_svm_linear.pkl', 'wb') as f:
    pickle.dump(clf, f)

Confusion matrix
 [[204   0   0]
 [ 82   3   1]
 [ 98   8  15]]
Confusion matrix
 [[203   0   0]
 [ 77   7   2]
 [107   7   7]]
Confusion matrix
 [[204   0   0]
 [ 78   8   0]
 [101  12   7]]
Confusion matrix
 [[204   0   0]
 [ 80   6   0]
 [ 96   7  17]]
Confusion matrix
 [[204   0   0]
 [ 78   6   1]
 [103   8  10]]
Acc:  0.5387609043973651
Precision:  [0.53102942 0.41256743 0.92487374]
Recall:  [1.         0.0699316  0.09289256]
